# MNIST-Azure
## Experiment Test
### By: Sebastian Goodfellow

In [1]:
# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import 3rd party libraries
import os
import sys
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.core import Workspace, Datastore, Experiment, RunConfiguration

# Import local Libraries
sys.path.insert(0, '/home/sebastiangoodfellow/Documents/Code/mnist-azure')
from mnistazure.config import DATA_PATH, TENSORBOARD_PATH

In [2]:
# Set input parameters
script_params = {'--data_dir': DATA_PATH,
                 '--log_dir': './logs',
                 '--batch_size': 32,
                 '--learning_rate': 1e-3,
                 '--epochs': 5,
                 '--max_to_keep': 1,
                 '--seed': 0}

In [3]:
# Get workspace
ws = Workspace.get(name='mnist-azure', subscription_id='', 
                   resource_group='')

In [4]:
# Get local run configuration
run_local = RunConfiguration()
run_local.environment.python.user_managed_dependencies = True

In [5]:
# Define experiment
exp = Experiment(workspace=ws, name='Experiment_5')

In [6]:
# Define estimator
estimator = Estimator(source_directory='../', compute_target='local',
                      entry_script='train.py', script_params=script_params,
                      environment_definition=run_local.environment)

In [ ]:
# Submit experiment
run = exp.submit(estimator, show_output=True)
run.wait_for_completion(show_output=True)